# 2 - Retrieve ERA5 Weather Data from Copernicus Climate Data Store

**This notebook will download ERA5 total precipitation weather data from the Copernicus Climate Data Store (CDS), covering the spatial and temporal extents of an RTC stack prepared for HydroSAR.**

---
## 0. Import required software

In [1]:
from pathlib import Path
import sys

import cdsapi
import ciso8601
from ipyfilechooser import FileChooser
import geopandas as gpd
import xarray as xr

current = Path("..").resolve()
sys.path.append(str(current))
import util.util as util

---
## 1. Select a directory holding an RTC stack prepared for HydroSAR

The directory should contain the following subdirectories:
- `DEM`
- `VV`

In [2]:
path = Path.home()
fc = FileChooser(path)
print('Select a data directory holding an RTC stack prepared for HydroSAR')
display(fc)

Select a data directory holding an RTC stack prepared for HydroSAR


FileChooser(path='/home/jovyan', filename='', title='', show_hidden=False, select_desc='Select', change_desc='…

---
## 2. Create a GeoDataFrame for the stack

In [3]:
data_path = Path(fc.selected_path)
vv_paths = sorted(list((data_path / 'VV').glob('*VV*.tif')))
epsg = 4326

gdf = gpd.GeoDataFrame(
    {
    'file': vv_paths,
    'SAR_acquisition_dt': util.get_dates(vv_paths),
    'geometry': [util.get_geotiff_bbox(pth, epsg) for pth in vv_paths]
    },
    crs=f'EPSG:{epsg}'
                  )
gdf

,file,SAR_acquisition_dt,geometry
0,/home/jovyan/bang/VV/S1A_IW_20180709T234718_DV...,2018-07-09 23:47:18,"POLYGON ((90.75509 23.61842, 93.59932 23.61842..."
1,/home/jovyan/bang/VV/S1A_IW_20180721T234719_DV...,2018-07-21 23:47:19,"POLYGON ((90.75509 23.61869, 93.59933 23.61869..."
2,/home/jovyan/bang/VV/S1A_IW_20180802T234719_DV...,2018-08-02 23:47:19,"POLYGON ((90.75597 23.61898, 93.60022 23.61898..."
3,/home/jovyan/bang/VV/S1A_IW_20180814T234720_DV...,2018-08-14 23:47:20,"POLYGON ((90.75509 23.61896, 93.59663 23.61896..."
4,/home/jovyan/bang/VV/S1A_IW_20180826T234721_DV...,2018-08-26 23:47:21,"POLYGON ((90.75480 23.61869, 93.59902 23.61869..."
5,/home/jovyan/bang/VV/S1A_IW_20180907T234721_DV...,2018-09-07 23:47:21,"POLYGON ((90.75450 23.61869, 93.59872 23.61869..."
6,/home/jovyan/bang/VV/S1A_IW_20180919T234722_DV...,2018-09-19 23:47:22,"POLYGON ((90.75480 23.61842, 93.59932 23.61842..."
7,/home/jovyan/bang/VV/S1A_IW_20181001T234722_DV...,2018-10-01 23:47:22,"POLYGON ((90.75568 23.61870, 93.60022 23.61870..."


## 3. Download the ERA5 precipitation data for the stack

In [4]:
# Create a folder to host the ERA5 data
weather_data_dir = data_path / 'ERA5'
weather_data_dir.mkdir(exist_ok=True)

# Create a new instance of the CDS API client
c = cdsapi.Client()

# # Create a list of the years covered by the tiffs
years = [str(dt.year) for dt in gdf['SAR_acquisition_dt']]
years = sorted(list(set(years)))
years

bounds = gdf.total_bounds.tolist()
bounds = [bounds[1], bounds[2], bounds[3], bounds[0]]

# Get the data from CDS
c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'variable': 'total_precipitation',
            'format': 'netcdf',
            'year': years,
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
            'area': bounds,  # AOI
        },
        f'{weather_data_dir}/era5_data.nc',
)

# Load precipitation dataset's name
filename = f'{weather_data_dir}/era5_data.nc'

# Load it, drop the 2nd level of precipitation (expver=5), resample for 1 value per day, in-between are summed, and save as a netcdf
ds = xr.open_dataset(filename).resample(time='D').sum().to_netcdf(f'{weather_data_dir}/era5_data.nc')

2024-05-10 21:53:22,424 INFO Welcome to the CDS
2024-05-10 21:53:22,425 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-10 21:53:22,705 INFO Request is completed
2024-05-10 21:53:22,705 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data8/adaptor.mars.internal-1715372718.7245975-6814-11-0dfb866d-03a8-4a5f-bcd1-0046cb00fe3c.nc to /home/jovyan/bang/ERA5/era5_data.nc (214.9M)
2024-05-10 21:54:09,818 INFO Download rate 4.6M/s    


*1b-ERA5_Download.ipynb - Version 1.0.0 - May 2024*